## 211014

### 라이브러리 로드

In [94]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import csv
from pprint import pprint # Json 출력 인덴트 주어 보기 편하게 출력.
import pandas as pd # 데이터프레임 가공을 위한 라이브러리.
import psycopg2 # 데이터베이스 연동을 위한 라이브러리.
from konlpy.tag import Twitter, Kkma # 한글 형태소 분석 위한 라이브러리.
import nltk # 영문 형태소 분석 + 명사, 동사 등 품사 구분 위한 라이브러리.
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\belab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
test = Twitter()
test.pos("저는 윤준 입니다")

[('저', 'Noun'), ('는', 'Josa'), ('윤준', 'Noun'), ('입니다', 'Adjective')]

### 엘라스틱 연동

In [4]:
es = Elasticsearch("")
es.info()

{'name': '6d37a750a0ac', 'cluster_name': 'docker-cluster', 'cluster_uuid': '', 'version': {'number': '7.3.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '4749ba6', 'build_date': '2019-08-19T20:19:25.651794Z', 'build_snapshot': False, 'lucene_version': '8.1.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}

### search 구문은 query from size sort 로 구성

In [70]:
# pprint(es.search(index='pheno', body={"query" : {"match_all" : {}}}))
test = es.search(index='pheno', body={"from":0, "size":1224,"query" : {"match_all" : {}}})
# pprint(test)

li = []
for i in range(0,1224):
    input_val = test['hits']['hits'][i]['_source']['pheno_origin']
    li.append(input_val)

<ipython-input-70-538cba4549b4>:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  test = es.search(index='pheno', body={"from":0, "size":1224,"query" : {"match_all" : {}}})


* pheno에서 원본 증/현상명 가져와 리스트에 저장

### count(*) 수행

In [38]:
es.search(index='pheno', body={
  "query": {
    "bool": {
      "must": []
    }
  },
  "_source": [
    "count"
  ]
})

<ipython-input-38-56fc07363d1d>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index='pheno', body={


{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1224, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'pheno', '_type': '_doc', '_id': '1', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '2', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '3', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '4', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '5', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '6', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '7', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '8', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '9', '_score': 1.0, '_source': {}}, {'_index': 'pheno', '_type': '_doc', '_id': '10', '_score': 1.0, '_source': {}}]}}

---------------------------
### PAPAGO API

client id : <br>
client pw : 

In [84]:
import requests
import os
import sys
import urllib.request
import json


def translate_with_papago(arg) :
    try :
        client_id = ""
        client_secret = ""
        encText = urllib.parse.quote(arg)
        data = "source=ko&target=en&text=" + encText
        url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode("utf-8"))
        rescode = response.getcode()
        
        response_body = response.read()
        res = json.loads(response_body.decode('utf-8'))
        tp = res['message']['result']['translatedText'] # 영문 추출.
        tp = del_dot(tp) # 영문 특수문자 제거.
        
        return tp # 반환.

    except :
        return ("Error Code:" + rescode)
    

# 특수문자 제거 함수.
def del_dot(arg) :
    tp = arg.replace('.','')
    tp = tp.replace('"','')
    tp = tp.split('/')[0].strip()
    return tp

### list -> dataframe 변환

In [79]:
df = pd.DataFrame(li, columns=['pheno_origin'])

### 영문 변환

In [85]:
df['pheno_origin_eng'] = df['pheno_origin'].apply(translate_with_papago)

### 형태소 분석 함수 생성

In [86]:
def eng_morpheme_analysis(arg) :
    tp = nltk.tag.pos_tag(nltk.tokenize.word_tokenize(arg))
    return tp

In [99]:
temp = df['pheno_origin_eng'].apply(eng_morpheme_analysis)

[('Dizziness', 'NN')]

In [105]:
temp.head(5)

0                                    [(Dizziness, NN)]
1                                        [(cough, NN)]
2                           [(Runny, NNP), (nose, NN)]
3    [(a, DT), (pain, NN), (in, IN), (the, DT), (gu...
4                                    [(Vomiting, VBG)]
Name: pheno_origin_eng, dtype: object

In [133]:
b=[]
for i in temp :
    a = []
    for j in i :        
        term = j[0]
        morpheme = j[-1]
        if morpheme in ('NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$', 'WP', 'WP$'):
            morpheme = '명사'
        elif morpheme in ('VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WP', 'WP$', 'MD'):
            morpheme = '동사'
        elif morpheme in ('JJ', 'JJR', 'JJS'):
            morpheme = '형용사'
        elif morpheme in ('RB', 'RBR', 'RBS'):
            morpheme = '부사'
        elif morpheme in ('IN'):
            morpheme = '전치사'
        tu = (term, morpheme)
        a.append(tu)
    b.append(a)

In [135]:
df['morpheme'] = b